In [5]:
import os
import pandas as pd

from beoid.gaze import read_gaze

In [6]:
gaze_data_root = os.path.join('/home/will/thesis/data/beoid-download/Bristol_Egocentric_Object_Interactions_Dataset_2014/TrackedGaze_2DAnd3D/')
gaze_file_suffix = '_3DGazeData.txt'
gaze_filenames = [file for file in os.listdir(gaze_data_root) if file.endswith(gaze_file_suffix)]
gaze_dfs = []
for filename in gaze_filenames:
    filepath = os.path.join(gaze_data_root, filename)
    video = filename[:-len(gaze_file_suffix)]
    gaze_df = read_gaze(filepath)
    gaze_df['Video'] = video
    gaze_dfs.append(gaze_df)
gaze = pd.concat(gaze_dfs)
valid_gaze = gaze.dropna(subset=['gaze_2d_x', 'gaze_2d_y', 'fixation_9'])

In [7]:
valid_gaze_descriptions = valid_gaze.groupby('Video').describe()
valid_gaze.describe()

,frame,gaze_2d_x,gaze_2d_y,camera_3d_x,camera_3d_y,camera_3d_z,gaze_3d_x,gaze_3d_y,gaze_3d_z,fixation_3,fixation_5,fixation_9
count,58977.000000,58977.000000,58977.000000,58977.000000,58977.000000,58977.000000,58977.000000,58977.000000,58977.000000,54983.0,55267.0,58977.0
mean,63508.805670,244.051858,217.014310,304.928659,304.373219,305.567448,304.989290,304.422270,305.147618,1.0,1.0,1.0
std,56274.237535,54.682298,97.370581,151.608472,151.661389,151.808139,151.616307,151.658657,151.941223,0.0,0.0,0.0
min,7.000000,2.790000,-0.980000,-0.242086,-1.152930,0.288874,-0.448802,-0.533981,-0.698502,1.0,1.0,1.0
25%,15115.000000,212.680000,142.010000,298.978000,300.038000,300.273000,298.979000,299.865000,299.984000,1.0,1.0,1.0
50%,29224.000000,241.520000,205.540000,399.458000,399.090000,400.795000,399.531000,399.297000,400.502000,1.0,1.0,1.0
75%,124176.000000,274.030000,289.450000,400.633000,399.469000,401.018000,400.705000,399.546000,400.697000,1.0,1.0,1.0
max,159736.000000,605.790000,479.740000,500.472000,500.829000,500.620000,500.349000,500.267000,500.512000,1.0,1.0,1.0


The source videos have a size 640 x 480 with no letterboxing, i.e. valid gaze data lies within the range ([0, 640], [0, 480]). We see in the above data that the `gaze_2d_{x,y}` sits mostly within these bounds, y just drops out under one.

In [8]:
def shift_frame(group):
    shifted_group = group.copy()
    shifted_group['frame'] = group['frame'] - group['frame'].min() + 1
    return shifted_group
    
shifted_gaze = gaze.groupby('Video').apply(shift_frame)
shifted_valid_gaze = shifted_gaze.dropna(subset=['gaze_2d_x', 'gaze_2d_y', 'fixation_9'])
shifted_valid_gaze.head()

time  frame  gaze_2d_x  gaze_2d_y  camera_3d_x  \
Video                                                               
00_Desk1 58  0:00:01.93     59     361.18     193.99      1.55657   
         61  0:00:02.03     62     297.24     195.59      1.49796   
         62  0:00:02.07     63     302.91     215.22      1.47920   
         63  0:00:02.10     64     304.02     213.47      1.46016   
         65  0:00:02.17     66     424.93     202.95      1.41186   

             camera_3d_y  camera_3d_z  gaze_3d_x  gaze_3d_y  gaze_3d_z  \
Video                                                                    
00_Desk1 58     -1.15293     0.637832   1.051400  -0.088111  -0.223826   
         61     -1.14502     0.647015   0.902907  -0.240535  -0.142056   
         62     -1.14734     0.650018   0.577920   0.267815  -0.678709   
         63     -1.14311     0.649413   0.894628  -0.221661  -0.195857   
         65     -1.13654     0.643896   1.208670  -0.205035  -0.060622   

             fixation_3  fixation_5  fixation_9     Video  
Video                                                      
00_Desk1 58         1.0         NaN         1.0  00_Desk1  
         61         NaN         1.0         1.0  00_Desk1  
         62         1.0         NaN         1.0  00_Desk1  
         63         1.0         NaN         1.0  00_Desk1  
         65         NaN         NaN         1.0  00_Desk1

In [ ]:
shifted_valid_gaze.to_pickle('../beoid-gaze-valid-shifted.pickle')